In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)
/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


#### Passengers

In [2]:
def create_cluster_script(categories):

    new_script_file = '/cellar/users/ramarty/Projects/hla_ii/data_gathering/mutations/patient_mutation_matrix.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(categories)))
        out_file.write("#$ -l h_vmem=8G\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set categories=({0})\n".format(" ".join(categories)))
        out_file.write("\n")

        out_file.write("set category=$categories[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("python /cellar/users/ramarty/Projects/hla_ii/data_gathering/mutations/patient_mutation_matrix.py $category\n")
        out_file.write("date\n")

In [3]:
create_cluster_script(['cgc_new'])

In [9]:
create_cluster_script(['passenger'])

#### Germline <br>
see: http://localhost:5000/notebooks/Projects/hla_new/post_submission/individual_analysis/germline_analysis.gather.ipynb

In [11]:
df = pd.read_csv('/cellar/users/ramarty/Data/hla/git_data/data/patient_affinities.germline.PHBR.csv',index_col=0)

In [12]:
df.shape

(3657, 1000)

In [14]:
def missesense_mutation(x):
    if x[0].isupper() and x[-1].isupper() and x[0] != x[-1]:
        return True
    else:
        return False
def merge_cols(x):
    return '{0}_{1}'.format(x[0], x[1])

In [24]:
all_dfs = []
for x in ['/cellar/users/ramarty/Data/hla/germline/Variant.Result.chr_1.tsv',
           '/cellar/users/ramarty/Data/hla/germline/Variant.Result.chr_2_4.tsv',
           '/cellar/users/ramarty/Data/hla/germline/Variant.Result.chr_5_7.tsv']:
    df = pd.read_csv(x, skiprows=11, sep='\t')
    all_dfs.append(df)
df = pd.concat(all_dfs)
df['valid'] = df['Protein sequence change'].apply(missesense_mutation)
df = df[df.valid]

In [27]:
df = df[['ID', 'dbSNP', 'HUGO symbol', 'Protein sequence change']]
df.columns = ['ID', 'dbSNP', 'gene', 'mutation']

In [31]:
df['valid'] = df.mutation.apply(missesense_mutation)
df['combined'] = df[['gene', 'mutation']].apply(merge_cols, axis=1)
df = df[df.valid]

In [33]:
df.head()

,ID,dbSNP,gene,mutation,valid,combined
1,1:909238,rs3829740,PLEKHN1,R487P,True,PLEKHN1_R487P
2,1:914333,rs13302979,C1orf170,E599Q,True,C1orf170_E599Q
3,1:914852,rs13303368,C1orf170,Q426E,True,C1orf170_Q426E
5,1:916549,rs6660139,C1orf170,W20R,True,C1orf170_W20R
6,1:935222,rs2298214,HES4,R44S,True,HES4_R44S


In [37]:
# reduce to those with affinity
mutations = [x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/presentation/residues/germline.txt').readlines()]
df = df[df.combined.isin(mutations)]

In [43]:
ids_w_affinity_ii = list(df.ID)
print len(ids_w_affinity_ii)

1000


In [45]:
with open('/cellar/users/ramarty/Data/hla/germline/ids_w_affinity_ii.txt', 'w') as out_file:
    for snp in ids_w_affinity_ii:
        out_file.write('{0}\n'.format(snp))

Use plink to get the patient associated variants <br>
cd /cellar/users/ramarty/Data/hla/germline <br>
/cellar/users/hcarter/programs/plink-1.9/plink -bfile imputed --snps-only --extract ids_w_affinity_ii.txt --recode A --out ids_w_affinity_ii

In [46]:
patient_mutation = pd.read_csv('/cellar/users/ramarty/Data/hla/germline/ids_w_affinity_ii.raw', sep=' ')

In [47]:
patient_mutation.index = patient_mutation.FID
patient_mutation = patient_mutation.ix[:, 6:]
patient_mutation.columns = list(mutations)

In [48]:
patient_mutation.shape

(4165, 1000)

In [50]:
columns = list(patient_mutation.columns)

In [51]:
def transform(x):
    if x == 2:
        return 1
    else:
        return x

In [53]:
for col in columns:
    patient_mutation[col] = patient_mutation[col].apply(transform)

In [55]:
patient_mutation.to_csv('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/germline.patient_mutations.csv', index_col=0)